In [1]:
from blackops.utils.catalog import start_spark_session
import pyspark.sql.functions as f
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
spark = start_spark_session()

24/11/03 02:26:46 WARN Utils: Your hostname, dadiego resolves to a loopback address: 127.0.1.1; using 192.168.104.128 instead (on interface eth1)
24/11/03 02:26:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/dadiego/projects/esic-bigdata-iv-blackops/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/dadiego/.ivy2/cache
The jars for the packages stored in: /home/dadiego/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
io.delta#delta-sharing-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-028ca85f-5d94-417b-951c-80780d9fe29e;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found io.delta#delta-sharing-spark_2.12;3.2.0 in central
	found io.delta#delta-sharing-client_2.12;1.0.5 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found commons-logging#commons-logging;1.2 in central
	found commons-codec#commons-codec;1.11 in central
:: resolution report :: resolve 399ms :: artifacts dl 18ms
	:: modules in use:
	commons-codec#commons-codec;1.11 from central in [default]
	commons-logging#commons-log

In [3]:
df = spark.read.csv("data/GroceryStoreDataSet.csv", schema="products string")
display(df)

products
"MILK,BREAD,BISCUIT"
"BREAD,MILK,BISCUIT,CORNFLAKES"
"BREAD,TEA,BOURNVITA"
"JAM,MAGGI,BREAD,MILK"
"MAGGI,TEA,BISCUIT"
"BREAD,TEA,BOURNVITA"
"MAGGI,TEA,CORNFLAKES"
"MAGGI,BREAD,TEA,BISCUIT"
"JAM,MAGGI,BREAD,TEA"
"BREAD,MILK"


In [4]:
df = df.select(f.split("products", ",").alias("products"))
display(df)

products
"[MILK, BREAD, BISCUIT]"
"[BREAD, MILK, BISCUIT, CORNFLAKES]"
"[BREAD, TEA, BOURNVITA]"
"[JAM, MAGGI, BREAD, MILK]"
"[MAGGI, TEA, BISCUIT]"
"[BREAD, TEA, BOURNVITA]"
"[MAGGI, TEA, CORNFLAKES]"
"[MAGGI, BREAD, TEA, BISCUIT]"
"[JAM, MAGGI, BREAD, TEA]"
"[BREAD, MILK]"


In [5]:
df.rdd.map(lambda x: x.products).collect()

[['MILK', 'BREAD', 'BISCUIT'],
 ['BREAD', 'MILK', 'BISCUIT', 'CORNFLAKES'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['JAM', 'MAGGI', 'BREAD', 'MILK'],
 ['MAGGI', 'TEA', 'BISCUIT'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['MAGGI', 'TEA', 'CORNFLAKES'],
 ['MAGGI', 'BREAD', 'TEA', 'BISCUIT'],
 ['JAM', 'MAGGI', 'BREAD', 'TEA'],
 ['BREAD', 'MILK'],
 ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'SUGER', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'COCK'],
 ['BREAD', 'SUGER', 'BISCUIT'],
 ['COFFEE', 'SUGER', 'CORNFLAKES'],
 ['BREAD', 'SUGER', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'SUGER'],
 ['BREAD', 'COFFEE', 'SUGER'],
 ['TEA', 'MILK', 'COFFEE', 'CORNFLAKES']]

In [6]:
te = TransactionEncoder()
data = te.fit_transform(df.toPandas()["products"]).astype("int")
df = pd.DataFrame(data, columns=te.columns_)
display(df)

/home/dadiego/projects/esic-bigdata-iv-blackops/.venv/lib/python3.11/site-packages/pyspark/sql/pandas/utils.py:24: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion


,BISCUIT,BOURNVITA,BREAD,COCK,COFFEE,CORNFLAKES,JAM,MAGGI,MILK,SUGER,TEA
0,1,0,1,0,0,0,0,0,1,0,0
1,1,0,1,0,0,1,0,0,1,0,0
2,0,1,1,0,0,0,0,0,0,0,1
3,0,0,1,0,0,0,1,1,1,0,0
4,1,0,0,0,0,0,0,1,0,0,1
5,0,1,1,0,0,0,0,0,0,0,1
6,0,0,0,0,0,1,0,1,0,0,1
7,1,0,1,0,0,0,0,1,0,0,1
8,0,0,1,0,0,0,1,1,0,0,1
9,0,0,1,0,0,0,0,0,1,0,0


In [7]:
prior = apriori(df, min_support = 0.2, use_colnames = True, verbose = 1)
display(prior)

Processing 42 combinations | Sampling itemset size 3


/home/dadiego/projects/esic-bigdata-iv-blackops/.venv/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.35,(BISCUIT)
1,0.20,(BOURNVITA)
2,0.65,(BREAD)
3,0.40,(COFFEE)
4,0.30,(CORNFLAKES)
5,0.25,(MAGGI)
6,0.25,(MILK)
7,0.30,(SUGER)
8,0.35,(TEA)
9,0.20,"(BISCUIT, BREAD)"


In [8]:
association_rules(prior, metric = "confidence", min_threshold = 0.6)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(MILK),(BREAD),0.25,0.65,0.2,0.800000,1.230769,0.0375,1.75,0.250000
1,(SUGER),(BREAD),0.30,0.65,0.2,0.666667,1.025641,0.0050,1.05,0.035714
2,(CORNFLAKES),(COFFEE),0.30,0.40,0.2,0.666667,1.666667,0.0800,1.80,0.571429
3,(SUGER),(COFFEE),0.30,0.40,0.2,0.666667,1.666667,0.0800,1.80,0.571429
4,(MAGGI),(TEA),0.25,0.35,0.2,0.800000,2.285714,0.1125,3.25,0.750000
